#### Packages

In [273]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt
from scipy.spatial.distance import squareform, pdist
import datetime
import math

#### Read Data

In [274]:
df = pd.read_csv('data\instance04.txt', header=None)

df_main = df.iloc[:2,:]
df_main.columns = df_main.iloc[0]
df_main = df_main[1:]
n_S = int(df_main['n_S'])
n_C = int(df_main['n_C'])
n_L = int(df_main['n_L'])
n_K = int(df_main['n_K'])
n_D = int(df_main['n_D'])
B = int(df_main['B'])//30

df_car =  df.iloc[3:n_C+4,:]
df_car.columns = df.iloc[3,:]
df_car = df_car.dropna(axis=1)
df_car = df_car[1:]
df_car = df_car.reset_index(drop=True)

car_current_station = []
for m in range(n_C):
    initial_station = int(df_car.loc[m, 'Initial station'])
    car_current_station.append(initial_station)
print("car_current_station: ", car_current_station)

df_car_lv = df.iloc[n_C+5:n_C+n_L+6,:]
df_car_lv.columns = df_car_lv.iloc[0]
df_car_lv = df_car_lv.dropna(axis=1)
df_car_lv = df_car_lv[1:]
df_car_lv = df_car_lv.reset_index(drop=True)

df_order = df.iloc[n_C+n_L+7:n_C+n_L+n_K+8,:]
df_order.columns = df_order.iloc[0]
df_order = df_order.dropna(axis=1)
df_order = df_order[1:]
df_order['Time span'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - pd.to_datetime(row['Pick-up time']), axis=1)
df_order['Time units'] = df_order['Time span'].apply(lambda x: math.ceil(x.total_seconds() / 1800))
first_period_start = pd.to_datetime('2023/01/01 00:00')
time_unit = pd.Timedelta('30 minutes')
df_order['Pick-up time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Pick-up time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order['Return time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order = pd.merge(df_order, df_car_lv, left_on='Level', right_on='Car level', how='left')
df_order = df_order.drop(['Car level'], axis=1)

# pickup station of order k
pickup = []
for k in range(n_K+1):
    if (k==0):
        pickup.append(int(0))
        continue
    pickup_station = int(df_order.loc[k-1, 'Pick-up station'])
    pickup.append(pickup_station)
# return station of order k
finish = []
for k in range(n_K+1):
    if (k==0):
        finish.append(int(0))
        continue
    return_station = int(df_order.loc[k-1, 'Return station'])
    finish.append(return_station)

df_distance = df.iloc[n_C+n_L+n_K+9:n_C+n_L+n_K+n_S**2+10,:]
df_distance.columns = df_distance.iloc[0]
df_distance = df_distance.dropna(axis=1)
df_distance = df_distance[1:]
distance = df_distance.pivot(index='From', columns='To', values='Distance')
distance = distance.apply(pd.to_numeric, errors='coerce') // 30
distance_array = distance.values
print(distance_array)
print("pickup station: ", pickup)
print("return station: ", finish)

lvD = []
for k in range(n_K+1):
    if (k==0):
        lvD.append(int(0))
        continue    
    level_demanded = int(df_order.loc[k-1, 'Level'])
    lvD.append(level_demanded)
# lvS_m = supplied car level of car m
lvS = []
for m in range(n_C):
    level_supplied = int(df_car.loc[m, 'Level'])
    lvS.append(level_supplied)

print("level_demanded: ", lvD)
print("level_supplied: ", lvS)

Cmax = 48*n_D

Revenue = []
for k in range(n_K+1):
    if (k==0):
        Revenue.append(int(0))
        continue
    time_units = int(df_order.loc[k-1, 'Time units'])
    hour_rate = int(df_order.loc[k-1, 'Hour rate'])
    Revenue.append(time_units*(hour_rate//2))
print("Revenue= ", Revenue)
print("Sum of revenue= ", sum(Revenue))

car_current_station:  [1, 4, 2, 9, 7, 3, 7, 6, 10, 7]
[[ 0 28 12 12 17 23 17 23 10 18]
 [28  0 16 18 11  9 15 21 20 16]
 [12 16  0 22 17 21 27 33  4 28]
 [12 18 22  0  7 13  7 11 22  8]
 [17 11 17  7  0  6 10 16 17 11]
 [23  9 21 13  6  0  6 12 21  7]
 [17 15 27  7 10  6  0  6 27  1]
 [23 21 33 11 16 12  6  0 33  5]
 [10 20  4 22 17 21 27 33  0 28]
 [18 16 28  8 11  7  1  5 28  0]]
pickup station:  [0, 7, 10, 10, 1, 7, 8, 6, 9, 8, 8, 2, 2, 1, 5, 4, 4, 9, 6, 8, 3]
return station:  [0, 3, 4, 2, 3, 10, 4, 9, 10, 1, 5, 8, 10, 8, 9, 5, 2, 10, 9, 4, 10]
level_demanded:  [0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]
level_supplied:  [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
Revenue=  [0, 2900, 1000, 2500, 2400, 10100, 2900, 7500, 3800, 7000, 7000, 2600, 3000, 5400, 12000, 26000, 16500, 12000, 9000, 7000, 8000]
Sum of revenue=  148600


#### Model

In [275]:
P1 = Model("P1")
#-------- Add variables as a list ---------#
# accept_k=1 if order k is accepted; accept_k=0 o.w.
accept = []
for k in range(n_K+1):
    if (k == 0):
        accept.append(int(1))
        continue
    accept.append(P1.addVar(lb=0, vtype=GRB.BINARY, name='accept_'+str(k)))

# z_mpq = 1 if order p and order q are on car m and order p is before order q
z = []
for m in range(n_C):
    z.append([])
    for p in range(n_K+1):
        z[m].append([])
        for q in range(n_K+1):
            z[m][p].append(P1.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(m+1) + str(p) + str(q)))

# Ck = completion time of order k
C = []
for k in range(n_K+1):
    if (k==0):
        C.append(int(0))
        continue   
    C.append(P1.addVar(lb=0, vtype = GRB.INTEGER, name = "C_" + str(k)))

# Om = latest completion time of orders on car m
O = []
for m in range(n_C):
    O.append(P1.addVar(lb=0, vtype = GRB.INTEGER, name = "O_" + str(m+1)))

# s_mpq: setup time for car m to process order q after finish order p
s = []
for m in range(n_C):
    s.append([])
    for p in range(n_K+1):
        s[m].append([])
        for q in range(n_K+1):
            s[m][p].append(P1.addVar(lb=0, vtype=GRB.INTEGER, name='s_'+str(m+1)+str(p)+str(q)))

# if there is no level-l's car in station i       
# uk = 1 if order k is upgraded
u = []
for k in range(n_K+1): 
    u.append(P1.addVar(lb = 0, vtype = GRB.BINARY, name = "u_" + str(k+1)))

# vk = 1 if order k is required to move a car from other station
v = []
for k in range(n_K+1):
    v.append(P1.addVar(lb = 0, vtype = GRB.BINARY, name = "v_" + str(k+1)))

# x_mk = time spent when accept order k assigned to car m (rental time)
x = []
for m in range(n_C):
    x.append([])
    for k in range(n_K+1):
        if (k==0):
            x[m].append(int(0))
            continue
        time_units = int(df_order.loc[k-1, 'Time units'])
        x[m].append(P1.addVar(lb = time_units, vtype = GRB.INTEGER, name = "x_" + str(m+1) + str(k)))

# y_mk = 1 if order k assigned to car m
y = []
for m in range(n_C):
    y.append([])
    for k in range(n_K+1):
        y[m].append(P1.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(m+1) + str(k)))

A = P1.addVar(lb = 100000, vtype = GRB.INTEGER, name = "a large number")

In [276]:
P1.setObjective(
    quicksum(accept[k]*Revenue[k] - (1-accept[k])*2*Revenue[k] for k in range(1, n_K+1)),
    GRB.MAXIMIZE)

In [277]:
# P1.addConstrs((2*z[m][p][q] <= accept[p] + accept[q] for p in range(n_K+1) for q in range(n_K+1) if (p!=q) for m in range(n_C)))
# Each order has a single predecessor (successor) on exactaly one of the car
P1.addConstrs((quicksum(quicksum(z[m][p][q] for m in range(n_C)) for p in range(n_K+1) if (p!=q))) == 1 for q in range(n_K))
P1.addConstrs((quicksum(quicksum(z[m][p][q] for m in range(n_C)) for q in range(n_K+1) if (p!=q))) == 1 for p in range(n_K))
# # An order can only have a predecessor on a car if it also has a successor on that same car
P1.addConstrs((quicksum(z[m][p][q] for q in range(n_K+1) if (p!=q)) == quicksum(z[m][h][p] for h in range(n_K+1) if (h!=p))) for p in range(n_K) for m in range(n_C))
# The completion times of each order s.t. if order p precedes order q on a car m, 
# the earliest time that order q can end must be greater than Cp + setup time from order p to order q and the processing time of order q
P1.addConstrs(C[q]-C[p] + A*(1-z[m][p][q]) >= s[m][p][q] + x[m][q] for p in range(n_K+1) for q in range(n_K) if (p!=q) for m in range(n_C))
# Only one order can be schedueled first on each car
P1.addConstrs(quicksum(z[m][0][p] for p in range(n_K)) <= 1 for m in range(n_C))

# C0=0, an auxiliary order used to enforce the start of the schedule
P1.addConstrs(C[0] == 0 for m in range(n_C))
# Calculate the makespan for each individual car
P1.addConstrs(quicksum(quicksum(((s[m][p][q] + x[m][q]) * z[m][p][q]) for q in range(n_K)) for p in range(n_K+1) if (p!=q))  == O[m] for m in range(n_C))
# Links the individual car makespan to the overall schedule makespan
P1.addConstrs((O[m] <= Cmax for m in range(n_C)))

# 移車/升等二選一
P1.addConstrs(u[k]+v[k] <= y[m][k] for k in range(n_K+1) for m in range(n_C)) 
# 如果有移車，那s[m][p][q]至少是7個單位(清潔*6 + 有移車要提早1單位到達) + 移車所需的時間
P1.addConstrs(s[m][p][q] >= 6+v[q]+distance_array[finish[p]-1][pickup[q]-1] for p in range(1, n_K+1) for q in range(1, n_K+1) if (p!=q) for m in range(n_C)) 
# 移車的總時間 <= B
P1.addConstrs(quicksum(accept[k] * v[k] * distance_array[finish[p]-1][pickup[k]-1] for k in range(1, n_K+1)) <= B  for p in range(1, n_K+1))

# 一個訂單最多分配給一台車
P1.addConstrs(quicksum(y[m][k] for m in range(n_C)) <= 1 for k in range(1, n_K+1))
P1.addConstrs((quicksum(y[m][k] for m in range(n_C)) <= quicksum(accept[k] for k in range(1, n_K+1)) for k in range(1, n_K+1))) 
# 接單的話，至少要提供跟客戶要求的level一樣的車
P1.addConstrs(y[m][k]*lvD[k] <= lvS[m] for k in range(1, n_K+1) for m in range(n_C))
# 接單的話，客戶領車的地方要有車
P1.addConstrs(y[m][k]*pickup[k] == y[m][k]*car_current_station[m] for k in range(1, n_K+1) for m in range(n_C))
P1.addConstrs(y[m][p] + y[m][q] <= z[m][p][q] + z[m][q][p] + 1 for m in range(n_C) for p in range(n_K+1) for q in range(n_K+1))
P1.update()

In [278]:
P1.optimize()
print("z* = ", P1.ObjVal)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9130 rows, 9323 columns and 42841 nonzeros
Model fingerprint: 0xbed011f9
Model has 4030 quadratic constraints
Variable types: 0 continuous, 9323 integer (4682 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  QMatrix range    [1e+00, 3e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e+03, 8e+04]
  Bounds range     [1e+00, 1e+05]
  RHS range        [1e+00, 2e+02]
  QRHS range       [2e+02, 2e+02]
Presolve removed 8880 rows and 702 columns
Presolve time: 0.15s
Presolved: 27860 rows, 44021 columns, 106220 nonzeros
Presolved model has 23600 SOS constraint(s)
Variable types: 0 continuous, 44021 integer (16200 binary)
Found heuristic solution: objective 148600.00000

Explored 1 nodes (0 simplex iterations) in 7.35 se